In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_sentiment.zip', 'train', 'test_images.zip', 'images', 'color_labels.csv', 'train_images.zip', 'test', 'train_metadata.zip', 'test_metadata.zip', 'breed_labels.csv', 'train_sentiment.zip', 'state_labels.csv']


In [4]:
train_df = pd.read_csv("../input/train/train.csv")

In [5]:
test_df = pd.read_csv("../input/test/test.csv")

In [6]:
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [7]:
drop_list = ['Name', 'RescuerID', 'Description']

In [8]:
train = train_df.drop(drop_list, axis=1)

In [9]:
test = test_df.drop(drop_list, axis=1)

In [10]:
x_train = train.drop(['PetID', 'AdoptionSpeed'], axis=1)

In [11]:
y_train = train['AdoptionSpeed']

In [13]:
from catboost import CatBoostClassifier, Pool

In [14]:
from sklearn.model_selection import StratifiedKFold, ParameterGrid, train_test_split

In [15]:
from sklearn.metrics import cohen_kappa_score

In [16]:
from tqdm import tqdm

In [17]:
RANDOM_STATE = 322

In [27]:
def cross_val(X, y, X_test, param, n_splits=3):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=RANDOM_STATE)
    
    acc = []
    predict = None
    
    for tr_ind, val_ind in skf.split(X, y):
        X_train = X[tr_ind]
        y_train = y[tr_ind]
        
        X_valid = X[val_ind]
        y_valid = y[val_ind]
        
        clf = CatBoostClassifier(iterations=500,
                                loss_function = param['loss_function'],
                                depth=param['depth'],
                                leaf_estimation_iterations = param['leaf_estimation_iterations'],
                                use_best_model=True,
                                logging_level='Verbose',
                                classes_count=5
                                
        )
        
        clf.fit(X_train, 
                y_train,
                eval_set=(X_valid, y_valid)
        )
        
        y_pred = clf.predict(X_valid)
        kappa = cohen_kappa_score(y_valid, y_pred)
        acc.append(kappa)
    return sum(acc)/n_splits

In [28]:
def catboost_GridSearchCV(X, y, X_test, params,n_splits=5):
    ps = {'acc':0,
          'param': []
    }
    
    predict=None
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        acc = cross_val(X, y, X_test, prms, n_splits=5)

        if acc>ps['acc']:
            ps['acc'] = acc
            ps['param'] = prms
    print('Acc: '+str(ps['acc']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

In [29]:
params = {'depth':[5,10,15],
          'leaf_estimation_iterations': [5,10,15],
          'loss_function': ['MultiClass']
    }

In [30]:
x_test = test.drop(['PetID'], axis=1)

In [ ]:
param = catboost_GridSearchCV(x_train.values, y_train.values, x_test, params)


Params Tuning::   0%|          | 0/9 [00:00<?, ?it/s]

0:	learn: -1.5935660	test: -1.5939514	best: -1.5939514 (0)	total: 74.1ms	remaining: 37s
1:	learn: -1.5791151	test: -1.5798139	best: -1.5798139 (1)	total: 174ms	remaining: 43.3s
2:	learn: -1.5655100	test: -1.5663899	best: -1.5663899 (2)	total: 250ms	remaining: 41.4s
3:	learn: -1.5541895	test: -1.5549128	best: -1.5549128 (3)	total: 337ms	remaining: 41.7s
4:	learn: -1.5429802	test: -1.5439767	best: -1.5439767 (4)	total: 409ms	remaining: 40.4s
5:	learn: -1.5322038	test: -1.5336080	best: -1.5336080 (5)	total: 486ms	remaining: 40s
6:	learn: -1.5223560	test: -1.5239429	best: -1.5239429 (6)	total: 563ms	remaining: 39.7s
7:	learn: -1.5137031	test: -1.5152464	best: -1.5152464 (7)	total: 641ms	remaining: 39.4s
8:	learn: -1.5054460	test: -1.5072403	best: -1.5072403 (8)	total: 734ms	remaining: 40s
9:	learn: -1.4975355	test: -1.4994592	best: -1.4994592 (9)	total: 821ms	remaining: 40.2s
10:	learn: -1.4902598	test: -1.4923303	best: -1.4923303 (10)	total: 894ms	remaining: 39.7s
11:	learn: -1.4829831	te

92:	learn: -1.3540556	test: -1.3653178	best: -1.3653178 (92)	total: 7.97s	remaining: 34.9s
93:	learn: -1.3537631	test: -1.3650930	best: -1.3650930 (93)	total: 8.05s	remaining: 34.8s
94:	learn: -1.3532084	test: -1.3646800	best: -1.3646800 (94)	total: 8.12s	remaining: 34.6s
95:	learn: -1.3528691	test: -1.3643833	best: -1.3643833 (95)	total: 8.2s	remaining: 34.5s
96:	learn: -1.3524978	test: -1.3641122	best: -1.3641122 (96)	total: 8.27s	remaining: 34.4s
97:	learn: -1.3520518	test: -1.3638795	best: -1.3638795 (97)	total: 8.4s	remaining: 34.5s
98:	learn: -1.3518228	test: -1.3637387	best: -1.3637387 (98)	total: 8.6s	remaining: 34.8s
99:	learn: -1.3514783	test: -1.3634390	best: -1.3634390 (99)	total: 8.71s	remaining: 34.9s
100:	learn: -1.3510943	test: -1.3632363	best: -1.3632363 (100)	total: 8.79s	remaining: 34.7s
101:	learn: -1.3503790	test: -1.3627451	best: -1.3627451 (101)	total: 8.87s	remaining: 34.6s
102:	learn: -1.3496062	test: -1.3621760	best: -1.3621760 (102)	total: 8.95s	remaining: 34

182:	learn: -1.3237311	test: -1.3468693	best: -1.3468693 (182)	total: 16.4s	remaining: 28.4s
183:	learn: -1.3235527	test: -1.3467662	best: -1.3467662 (183)	total: 16.5s	remaining: 28.3s
184:	learn: -1.3232338	test: -1.3465863	best: -1.3465863 (184)	total: 16.5s	remaining: 28.1s
185:	learn: -1.3231210	test: -1.3465501	best: -1.3465501 (185)	total: 16.6s	remaining: 28.1s
186:	learn: -1.3229025	test: -1.3464706	best: -1.3464706 (186)	total: 16.7s	remaining: 28s
187:	learn: -1.3225572	test: -1.3464360	best: -1.3464360 (187)	total: 16.8s	remaining: 27.9s
188:	learn: -1.3223669	test: -1.3462828	best: -1.3462828 (188)	total: 16.9s	remaining: 27.8s
189:	learn: -1.3219833	test: -1.3461220	best: -1.3461220 (189)	total: 17s	remaining: 27.7s
190:	learn: -1.3216536	test: -1.3458745	best: -1.3458745 (190)	total: 17s	remaining: 27.6s
191:	learn: -1.3214055	test: -1.3457986	best: -1.3457986 (191)	total: 17.1s	remaining: 27.5s
192:	learn: -1.3212023	test: -1.3456709	best: -1.3456709 (192)	total: 17.2s	

271:	learn: -1.3045878	test: -1.3380691	best: -1.3380691 (271)	total: 24s	remaining: 20.1s
272:	learn: -1.3044452	test: -1.3380343	best: -1.3380343 (272)	total: 24.1s	remaining: 20s
273:	learn: -1.3041020	test: -1.3379261	best: -1.3379261 (273)	total: 24.1s	remaining: 19.9s
274:	learn: -1.3039904	test: -1.3379299	best: -1.3379261 (273)	total: 24.2s	remaining: 19.8s
275:	learn: -1.3038997	test: -1.3378716	best: -1.3378716 (275)	total: 24.3s	remaining: 19.7s
276:	learn: -1.3036442	test: -1.3377496	best: -1.3377496 (276)	total: 24.4s	remaining: 19.6s
277:	learn: -1.3035683	test: -1.3376976	best: -1.3376976 (277)	total: 24.5s	remaining: 19.6s
278:	learn: -1.3034384	test: -1.3375774	best: -1.3375774 (278)	total: 24.6s	remaining: 19.5s
279:	learn: -1.3033089	test: -1.3375026	best: -1.3375026 (279)	total: 24.7s	remaining: 19.4s
280:	learn: -1.3031550	test: -1.3374190	best: -1.3374190 (280)	total: 24.8s	remaining: 19.3s
281:	learn: -1.3030327	test: -1.3372897	best: -1.3372897 (281)	total: 24.8

...

253:	learn: -1.1168585	test: -1.3249558	best: -1.3249558 (253)	total: 53m 58s	remaining: 52m 16s
254:	learn: -1.1165426	test: -1.3249363	best: -1.3249363 (254)	total: 53m 58s	remaining: 51m 51s
255:	learn: -1.1157974	test: -1.3246902	best: -1.3246902 (255)	total: 53m 58s	remaining: 51m 26s
256:	learn: -1.1155304	test: -1.3246415	best: -1.3246415 (256)	total: 53m 58s	remaining: 51m 2s
257:	learn: -1.1119574	test: -1.3243675	best: -1.3243675 (257)	total: 54m 17s	remaining: 50m 55s
258:	learn: -1.1117807	test: -1.3243997	best: -1.3243675 (257)	total: 54m 18s	remaining: 50m 31s
259:	learn: -1.1107382	test: -1.3241983	best: -1.3241983 (259)	total: 54m 19s	remaining: 50m 8s
260:	learn: -1.1107095	test: -1.3241982	best: -1.3241982 (260)	total: 54m 19s	remaining: 49m 44s
261:	learn: -1.1092652	test: -1.3242013	best: -1.3241982 (260)	total: 56m 24s	remaining: 51m 14s
262:	learn: -1.1092449	test: -1.3241798	best: -1.3241798 (262)	total: 56m 24s	remaining: 50m 50s
263:	learn: -1.1088429	test: -1.

337:	learn: -1.0637474	test: -1.3216243	best: -1.3216243 (337)	total: 1h 13m 44s	remaining: 35m 20s
338:	learn: -1.0635371	test: -1.3216239	best: -1.3216239 (338)	total: 1h 13m 44s	remaining: 35m 1s
339:	learn: -1.0633520	test: -1.3215253	best: -1.3215253 (339)	total: 1h 14m 53s	remaining: 35m 14s
340:	learn: -1.0633224	test: -1.3214978	best: -1.3214978 (340)	total: 1h 14m 53s	remaining: 34m 55s
341:	learn: -1.0626921	test: -1.3215311	best: -1.3214978 (340)	total: 1h 14m 56s	remaining: 34m 37s
342:	learn: -1.0609421	test: -1.3215562	best: -1.3214978 (340)	total: 1h 16m 28s	remaining: 35m
343:	learn: -1.0607954	test: -1.3215571	best: -1.3214978 (340)	total: 1h 16m 29s	remaining: 34m 41s
344:	learn: -1.0601306	test: -1.3213863	best: -1.3213863 (344)	total: 1h 16m 29s	remaining: 34m 22s
345:	learn: -1.0598070	test: -1.3213768	best: -1.3213768 (345)	total: 1h 18m 2s	remaining: 34m 44s
346:	learn: -1.0591660	test: -1.3214790	best: -1.3213768 (345)	total: 1h 18m 26s	remaining: 34m 35s
347:	l

420:	learn: -1.0263045	test: -1.3201948	best: -1.3201948 (420)	total: 1h 44m 25s	remaining: 19m 35s
421:	learn: -1.0259555	test: -1.3201857	best: -1.3201857 (421)	total: 1h 44m 26s	remaining: 19m 18s
422:	learn: -1.0247129	test: -1.3204062	best: -1.3201857 (421)	total: 1h 44m 51s	remaining: 19m 5s
423:	learn: -1.0234177	test: -1.3201096	best: -1.3201096 (423)	total: 1h 46m 34s	remaining: 19m 6s
424:	learn: -1.0230021	test: -1.3201026	best: -1.3201026 (424)	total: 1h 48m 9s	remaining: 19m 5s
425:	learn: -1.0219947	test: -1.3199145	best: -1.3199145 (425)	total: 1h 49m 28s	remaining: 19m
426:	learn: -1.0217538	test: -1.3199689	best: -1.3199145 (425)	total: 1h 49m 29s	remaining: 18m 43s
427:	learn: -1.0217503	test: -1.3199690	best: -1.3199145 (425)	total: 1h 49m 29s	remaining: 18m 25s
428:	learn: -1.0214437	test: -1.3199429	best: -1.3199145 (425)	total: 1h 49m 29s	remaining: 18m 7s
429:	learn: -1.0200730	test: -1.3200135	best: -1.3199145 (425)	total: 1h 49m 55s	remaining: 17m 53s
430:	lear

3:	learn: -1.5478192	test: -1.5516047	best: -1.5516047 (3)	total: 5.22s	remaining: 10m 46s
4:	learn: -1.5361631	test: -1.5403318	best: -1.5403318 (4)	total: 5.33s	remaining: 8m 47s
5:	learn: -1.5243091	test: -1.5289087	best: -1.5289087 (5)	total: 5.49s	remaining: 7m 32s
6:	learn: -1.5118004	test: -1.5187488	best: -1.5187488 (6)	total: 8.44s	remaining: 9m 54s
7:	learn: -1.5015164	test: -1.5092329	best: -1.5092329 (7)	total: 8.81s	remaining: 9m 1s
8:	learn: -1.4915305	test: -1.5001220	best: -1.5001220 (8)	total: 9.16s	remaining: 8m 19s
9:	learn: -1.4844003	test: -1.4932592	best: -1.4932592 (9)	total: 9.24s	remaining: 7m 32s
10:	learn: -1.4744419	test: -1.4850441	best: -1.4850441 (10)	total: 12.1s	remaining: 8m 57s
11:	learn: -1.4674200	test: -1.4782617	best: -1.4782617 (11)	total: 12.2s	remaining: 8m 16s
12:	learn: -1.4598306	test: -1.4716490	best: -1.4716490 (12)	total: 12.9s	remaining: 8m 3s
13:	learn: -1.4506579	test: -1.4647070	best: -1.4647070 (13)	total: 1m 48s	remaining: 1h 2m 46s

89:	learn: -1.2814110	test: -1.3555982	best: -1.3555982 (89)	total: 16m 28s	remaining: 1h 15m 1s
90:	learn: -1.2793117	test: -1.3551419	best: -1.3551419 (90)	total: 16m 28s	remaining: 1h 14m 2s
91:	learn: -1.2783324	test: -1.3549126	best: -1.3549126 (91)	total: 16m 28s	remaining: 1h 13m 4s
92:	learn: -1.2775525	test: -1.3546905	best: -1.3546905 (92)	total: 16m 28s	remaining: 1h 12m 7s
93:	learn: -1.2774391	test: -1.3546182	best: -1.3546182 (93)	total: 16m 28s	remaining: 1h 11m 10s
94:	learn: -1.2768232	test: -1.3544937	best: -1.3544937 (94)	total: 16m 29s	remaining: 1h 10m 16s
95:	learn: -1.2739139	test: -1.3539282	best: -1.3539282 (95)	total: 16m 30s	remaining: 1h 9m 28s
96:	learn: -1.2713914	test: -1.3534861	best: -1.3534861 (96)	total: 18m 21s	remaining: 1h 16m 16s
97:	learn: -1.2673007	test: -1.3529246	best: -1.3529246 (97)	total: 19m 18s	remaining: 1h 19m 11s
98:	learn: -1.2643103	test: -1.3525055	best: -1.3525055 (98)	total: 20m 54s	remaining: 1h 24m 40s
99:	learn: -1.2636710	tes

In [ ]:
print(param)

In [ ]:
x_test = test.drop(['PetID'], axis=1)

In [ ]:
param = catboost_GridSearchCV(x_train, y_train, x_test, params)


In [ ]:
test_pet_id = test['PetID']

In [ ]:
pred_test = model.predict(x_test)

In [ ]:
pred = list(map(lambda x: int(x[0]), pred_test))

In [ ]:
s = set(pred)

In [ ]:
s

In [ ]:
submit_df = pd.DataFrame(data={'PetID': test_pet_id, 'AdoptionSpeed': pred})

In [ ]:
submit_df.head()

In [ ]:
submit_df.to_csv('submission.csv', index=False)